# 直接基于brant的统计mat进行的计算，（网络指标）和临床指标的相互关系(pair时间点)

In [ ]:
#分开进行分析，得到图
import os
import shutil
import seaborn as sns
import matplotlib.pyplot as plt
from PDdatasets.subject import load_subjects
import numpy as np
import pandas as pd
from statistical_analysis import correlation

subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')


threshold_value = 0.3 #阈值
out_path = r'H:\006pd_DTI\11_subnetwork_brant\12_4.0base360dsubnetwork\sta' #输出路径

#metrics = ['degree','global_efficiency', 'transitivity']

metrics = ['degree']

c_names = ['handtremor','CRSTA_total','CRST b_total','CRST C','CRST TOTAL']

for metric in metrics:
    for c_name in c_names:
        subjects_values = []
        clinical_values = []
        for subject in subjects:
            observations = subject.get_all_observation()
            for obs in observations:
                if obs.name != 'ncnc' and obs.name != '001d'and obs.name != '007d':
                    value = obs.bold.get_global_metric(metric, mat_name='brant_post/4.0base360dsubnetwork_network.mat', #对应的NBS文件
                                                    threshold_type='intensity', #阈值类型
                                                    threshold_value=threshold_value)
                    subjects_values.append(float(value))
                    clinical_values.append(obs.args[c_name])

        r, p = correlation.correlation(subjects_values, clinical_values, show=False)

        _, ax = plt.subplots(figsize=(float(5), float(5)))
        ax = sns.regplot(x=subjects_values,y=clinical_values, robust=True,
                            ax=ax,  scatter=False)
        ax.set_title('r={:.2f}, p={:.2e}'.format(r, p), fontdict={'fontsize': 14})

        ax.set_ylim(bottom=0, top=80)
        ax.tick_params(axis='both', which='major', labelsize=14)
        plt.tight_layout()
        plt.show()

        print('{}-{}, r:{}, p:{}'.format(metric, c_name, r, p))


# 直接基于brant的统计mat进行的计算，（节点指标）和临床指标的相互关系（pair时间点）

In [ ]:
from PDdatasets.subject import load_subjects
from statistical_analysis import ttest
from scipy.stats import ttest_rel
from statistical_analysis.ttest import load_clinical
from statistical_analysis import correlation, plsr
from PDutils import NBS_subnet_extract
import pandas as pd
import numpy as np
from scipy.stats import ttest_rel

from statistical_analysis.ttest import load_subjects_nodal_metric

obs_name1 = 'base'
obs_name2 = '030d'
intensity_thres = 0.3
mat_name = 'brant_post/03basecombine_network.mat'

subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')

"""
# Readin NBS defined subnetwork
NBS_path1 = r'H:\006pd_DTI\13_C_BOLD_NBS_NEW\60_SM_R_weight_it0.3___\ncbase____\weight_start_net.mat'
subnetwork1 = subnetwork_extract.readin_NBS_subnetwork(NBS_path1)
"""

# Readin txt defined subnetwork
subnetwork1 = np.loadtxt(r'H:\006pd_DTI\10_BN_nbs_bold\02_R_wi0.3\combine.txt', delimiter =',')
all_nodes, row_nodes, col_nodes = NBS_subnet_extract.extract_subnetwork_nodes(subnetwork1) # matlab中坐标是从1开始
all_nodes = all_nodes+1

metrics = ['neighbor_degree',
            'betweenness_centrality', 'degree',
            'fault_tolerance', 'shortest_path_length',
            'global_efficiency', 'clustering_coefficient',
            'local_efficiency', 'vulnerability']


metric_v1, metric_v2 = load_subjects_nodal_metric(subjects, obs_name1, obs_name2, mat_name, 'degree', thres_type, intensity_thres)
ts, ps = ttest_rel(metric_v1, metric_v2, axis=1)

value_dict = dict(zip(all_nodes, ts))

resutls = plsr.plsr(value_dict, n_components=2,
                n_perm=1, n_boot=1,
                gene_path=r'H:\006pd_DTI\05_gene_analysis\01_AHBAexpression\expression.csv',
                out_csv_path=r'H:\006pd_DTI\temp1/plsr.csv',
                out_model_path=r'H:\006pd_DTI\temp1/plsr.pickle')
print(resutls.permres.pvals)


In [ ]:
from PDdatasets.subject import load_subjects
from statistical_analysis import ttest
from scipy.stats import ttest_rel
from statistical_analysis.ttest import load_clinical
from statistical_analysis import correlation
from PDutils import NBS_subnet_extract
import pandas as pd
import numpy as np
from statistical_analysis.ttest import load_subjects_nodal_metric

obs_name1 = 'base'
obs_name2 = '360d'
thres_type ='intensity'
intensity_thres = 0.3
out_dir = r'H:\006pd_DTI\temp1'

subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')

mat_name = 'brant_post/wholenet_network.mat'

metrics = ['degree','global_efficiency', 'clustering_coefficient']

c_names = ['handtremor','CRST_A','CRST_B', 'CRST_C', 'CRST_TOTAL' ]

for c_name in c_names:
    clinical_v1 = load_clinical(subjects, obs_name1, c_name)
    clinical_v2 = load_clinical(subjects, obs_name2, c_name)
    clinical_delta = clinical_v1 - clinical_v2

    for metric in metrics:
        metric_v1, metric_v2 = load_subjects_nodal_metric(subjects, obs_name1, obs_name2, mat_name, metric, thres_type, intensity_thres)
        metric_delta = metric_v1 - metric_v2

        i = 0
        for node_delta in metric_delta:
            r, p = correlation.correlation(clinical_delta, node_delta)
            if p < 2.04e-04:
                #correlation.correlation(clinical_delta, metric_delta, save=True, out_path=os.path.join(out_dir, '{}_{}'.format(metric, c_name)))
                print('{}_{}: node:{}, r:{:.4f} p:{:.4e}'.format(c_name, metric, i, r, p))
            i += 1

# 在（全时序）和临床相关分析后，(网络指标)和临床指标的相互关系汇总图，分开展示版+汇总展示版

In [ ]:
from PDdatasets.subject import load_subjects
from statistical_analysis import correlation
import numpy as np
from statistical_analysis.ttest import load_subjects_nodal_metric
import os

subjects = load_subjects(r'H:\006pd_DTI\02_data_management\subject_info.csv')
#out_dir = r"H:\006pd_DTI\14_nodal_analysis"

from statistical_analysis.ttest import load_clinical

obs_name1 = 'base'
obs_name2 = '360d'

mat_name = 'brant_post/wholenet_network.mat'
thres_type = 'intensity'
threshold_value = 0.3

c_names = ['handtremor','CRST_A','CRST_B', 'CRST_C', 'CRST_TOTAL' ]

metrics = ['degree','global_efficiency', 'clustering_coefficient']

for c_name in c_names:
    clinical_v1 = load_clinical(subjects, obs_name1, c_name)
    clinical_v2 = load_clinical(subjects, obs_name2, c_name)
    clinical_delta = clinical_v1 - clinical_v2

    for m_name in metrics:
        metric_v1, metric_v2 = load_subjects_nodal_metric(subjects, obs_name1, obs_name2, mat_name, m_name, thres_type, threshold_value)
        delta_metric = metric_v1 - metric_v2

        j = 1
        for v in delta_metric:
            r,p = correlation.correlation(clinical_delta, v)
            if p < 0.05:
                print('Clinical:{} obs:{}-{}, metric:{}, Node:{}, r:{:.2f}, p:{:.2e}'.format(c_name, obs_name1, obs_name2, m_name, j, r, p))
                #correlation.correlation(clinical_delta, v, save=True, out_path=os.path.join(out_dir, 'Clinical:{} obs:{}-{}, metric:{}, Node:{}, r:{:.2f}, p:{:.2e}'.format(c_name, obs_name1, obs_name2, m_name, j, r, p)))
            j += 1

